In [1]:
!pip install transformers
!pip install datasets
!pip install jiwer

# **Generate CSV for Submission**

In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import os
import pandas as pd
import torchaudio
import torch
from datasets import load_metric

# Parameters
language_code = 'ar'
language_name = 'arabic'
new_output_models_dir = f"/home/nmu1/Downloads/MTC/Train Model/30-2/output_models_Demo30_2/ar/wav2vec2-large-xlsr-arabic/finetuned-v1/"
test_data_path = "/home/nmu1/Downloads/MTC/Dataset/test"

# Load WER metric, processor, and model
wer_metric = load_metric("wer")
processor = Wav2Vec2Processor.from_pretrained(new_output_models_dir)
model = Wav2Vec2ForCTC.from_pretrained(new_output_models_dir)
model.to("cuda")

# Function to transcribe audio file
def transcribe(audio_file_path):
    waveform, sample_rate = torchaudio.load(audio_file_path)
    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values.to("cuda")
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Generate transcription for all test files
submission_data = []
for audio_file in os.listdir(test_data_path):
    if audio_file.endswith('.wav'):
        audio_file_path = os.path.join(test_data_path, audio_file)
        transcription = transcribe(audio_file_path)
        audio_file_id = os.path.splitext(audio_file)[0]
        submission_data.append((audio_file_id, transcription))

# Create DataFrame
submission_df = pd.DataFrame(submission_data, columns=['audio', 'transcript'])

# Save to CSV
submission_file_path = os.path.join(new_output_models_dir, 'submission_1.csv')
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
2024-06-26 10:13:01.023387: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 10:13:01.552128: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failin

Submission file saved to /home/nmu1/Downloads/MTC/Train Model/30-2/output_models_Demo30_2/ar/wav2vec2-large-xlsr-arabic/finetuned-v1/submission_1.csv
